<a href="https://colab.research.google.com/github/alinelena/janus-lavello/blob/main/hello_janus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install dependencies, pay special atention to install ase by hand

In [ ]:
!python3 -m pip install git+https://github.com/stfc/janus-core.git
!python3 -m pip install -U git+https://gitlab.com/ase/ase.git

In [ ]:
from ase.build import bulk
from ase.io import write

from janus_core.single_point import SinglePoint

NaCl = bulk('NaCl', 'rocksalt', a=5.63, cubic=True)
write("NaCl.xyz",NaCl)
sp = SinglePoint(structure="NaCl.xyz",
                 architecture="mace_mp",
                 device='cpu',
                 calc_kwargs={'model_paths':'medium'})

res = sp.run_single_point()
print(res)

In [ ]:
from ase.build import molecule
from ase.io import write

from janus_core.single_point import SinglePoint

h2o = molecule('H2O')
write("H2O.xyz",h2o)
sp = SinglePoint(structure="H2O.xyz",
                 architecture="mace_mp",
                 device='cpu')

res = sp.run_single_point()
print(res)